### Packages

In [1]:
using DynamicalSystems, DifferentialEquations, OrdinaryDiffEq, StaticArrays
using CairoMakie

### Functions

In [50]:
function model(u, p ,t)
    
    E, x, u_, y = u
    τ, α, τ_D, J, U0, ΔU0, τ_y, β, xthr, ythr, I0 = p
    
    U(y, U0, ΔU0, ythr) = U0 + ΔU0 / ( 1 + exp( -50 * ( y - ythr ) ) )
    σ(x, xthr)= 1 / (1 + exp(-20 * (x - xthr)));
    
    du1 = (-E + α * log( 1 + exp( ( J * u_ * x * E + I0 ) / α ) )) / τ
    du2 = ( 1 - x ) / τ_D - u_ * x * E
    du3 = (U(y, U0, ΔU0, ythr) - u_) / τ_F + U(y, U0, ΔU0, ythr) * ( 1 - u_ ) * E
    du4 = -y /  τ_y + β * σ(x, xthr)
    
    return SVector(du1, du2, du3, du4)
    
end;

function output(index_i, index_j, U0_lc, I0_lc, u0_lc)
        println("index i: $index_i"); flush(stdout)
        println("index j: $index_j"); flush(stdout)
        println("U0: $U0_lc;"); flush(stdout)
        println("I0: $I0_lc;"); flush(stdout)
        println("u0: $u0_lc;"); flush(stdout)
end
function separate()
    println(" --------------------- "); flush(stdout)
    println("  "); flush(stdout)
end

function Y(x)
    a = 0.1836121365412579
    b = 0.2749462902187088
    c = 0.25
    return  a * (x.^2) + b * x .+ c 
end

function print_output(index, I0, U0, initial_condition)
    println("index: $index");flush(stdout)
    println("I0: $I0");flush(stdout)
    println("U0: $U0");flush(stdout)
    println("initial condition: $initial_condition")
end
function separate()
    println("_________________")
    println("")
end

separate (generic function with 1 method)

### Parametres

In [45]:
τ = 0.013; τ_D = 0.15; τ_y = 1.8; τ_F = 1;
α = 1.5; J = 3.07; β = 0.4375
ΔU0 = 0.305
xthr = 0.9; ythr = 0.5
parametres = SA[ τ, α, τ_D, J, NaN, ΔU0, τ_y, β, xthr, ythr, NaN ]
timefinal = 500.0
println(parametres)

[0.013, 1.5, 0.15, 3.07, NaN, 0.305, 1.8, 0.4375, 0.9, 0.5, NaN]


### Grid

In [86]:
startI = -1.8
endI = -1.24798
lengthI = 100

rangeI = range( startI, endI, length = lengthI )
rangeU0 = Y( rangeI );

In [87]:
file = "output_from_function.txt"

"output_from_function.txt"

In [ ]:
for (index, I0) in enumerate(rangeI)
    """
        Итерируемся по точкам c функции
    """
    if index == 1
        global initial_condition_function = SA[0.0, 0.2, 0.1, 0.1]
    end
    
    U0 = Y(I0)
    
    print_output(index, I0, U0, initial_condition_function)
    
    parametres = SA[  τ, α, τ_D, J, U0, ΔU0, τ_y, β, xthr, ythr, I0  ]
    ds = ContinuousDynamicalSystem( model, initial_condition_function, parametres )
    
    tr = trajectory(ds, timefinal, Ttr = timefinal, Δt = 0.001;
                    diffeq = (alg = RK4(), adaptive = false, dense = false, dt = 0.001))
    
    initial_condition_function = tr[end, :]
    println("last point: $initial_condition_function");flush(stdout)
    λλ = lyapunovspectrum( ds, timefinal, Ttr = timefinal, Δt = 1;
                            diffeq = ( alg = RK4(), adaptive = false, dense = false, dt = 0.001 ))
    println("LSE: $λλ");flush(stdout)
    """
        this save output
    """
    open(file, "a") do io
        println(io, " $I0 $U0 $(λλ[1])")
    end
    
    firstdot( I0, U0, initial_condition_function )
    
    separate()
end

index: 1
I0: -1.8
U0: 0.34999999999999987
initial condition: [0.0, 0.2, 0.1, 0.1]
last point: [1.9774203443433476, 0.8191410397203346, 0.7440336651009856, 0.039499383434498825]
LSE: [-0.00022855632034940893, -0.555555539508975, -2.278804953508714, -8.775049309549207]
FIRST ITERATION
MOVE TO RIGHT
->
index: 1
I0: -1.7944240404040406
U0: 0.34999999999999987
initial condition: [1.9774203443433476, 0.8191410397203346, 0.7440336651009856, 0.039499383434498825]
last point: [1.2803655145184345, 0.7987592975681482, 0.7613312732237268, 0.035125350692809296]
LSE: [0.001484303391490763, -0.5555555556801215, -2.321858223557145, -8.652236194997045]
_________________

->
index: 2
I0: -1.788848080808081
U0: 0.34999999999999987
initial condition: [1.2803655145184345, 0.7987592975681482, 0.7613312732237268, 0.035125350692809296]
last point: [0.9759435544199823, 0.7366430687949771, 0.7856605876013553, 0.032568412554895565]
LSE: [0.001152579417384278, -0.5555555561057685, -2.3640379390914816, -8.52091156

LSE: [0.007573097907152285, -0.5555555553781446, -2.9263973189912384, -4.978713115052772]
_________________

->
index: 23
I0: -1.6717529292929323
U0: 0.34999999999999987
initial condition: [28.29861655726123, 0.4821133760128753, 0.8078020781678754, 0.01338939113461118]
last point: [1.3552962832380655, 0.6073759063763853, 0.8219657263345472, 0.01129353314478279]
LSE: [0.0015223754392209164, -0.5555555555318348, -2.936281446343415, -4.743891778335737]
_________________

->
index: 24
I0: -1.6661769696969728
U0: 0.34999999999999987
initial condition: [1.3552962832380655, 0.6073759063763853, 0.8219657263345472, 0.01129353314478279]
last point: [1.897485211487522, 0.7519022330348587, 0.7818140923494131, 0.010935935202837456]
LSE: [-0.003883562088016065, -0.5555555561464718, -2.939121246598829, -4.503414429785062]
_________________

->
index: 25
I0: -1.6606010101010134
U0: 0.34999999999999987
initial condition: [1.897485211487522, 0.7519022330348587, 0.7818140923494131, 0.010935935202837456]


initial condition: [9.883964084344889, 0.44121052317352283, 0.8542400699058622, 8.150999111040266e-5]
last point: [9.897122860432376, 0.44083914588207007, 0.8543904527130821, 8.09068767186309e-5]
LSE: [-0.5555555555687742, -1.720669371977529, -1.7201063870782034, -4.243148403648088]
_________________

->
index: 46
I0: -1.5435058585858645
U0: 0.34999999999999987
initial condition: [9.897122860432376, 0.44083914588207007, 0.8543904527130821, 8.09068767186309e-5]
last point: [9.910253175081097, 0.4404691688045531, 0.8545402012731571, 8.031047325368497e-5]
LSE: [-0.555555555568774, -1.7526545411423917, -1.7577112938010353, -4.248792762849032]
_________________

->
index: 47
I0: -1.537929898989905
U0: 0.34999999999999987
initial condition: [9.910253175081097, 0.4404691688045531, 0.8545402012731571, 8.031047325368497e-5]
last point: [9.923355273921622, 0.4401005798066231, 0.8546893213341025, 7.972067863834469e-5]
LSE: [-0.555555555568774, -1.7902203403940549, -1.7895411329846902, -4.25441967

LSE: [-0.5555555555687541, -2.465141575860881, -2.463822830804214, -4.3635476927815215]
_________________

->
index: 68
I0: -1.4208347474747562
U0: 0.34999999999999987
initial condition: [10.17982183189432, 0.4330030149512341, 0.8575479107744479, 6.917165077693315e-5]
last point: [10.192382587369362, 0.4326610475950516, 0.857685027320737, 6.870021710460025e-5]
LSE: [-0.5555555555687541, -2.49789257366138, -2.4966953716068487, -4.3688452107313145]
_________________

->
index: 69
I0: -1.4152587878787968
U0: 0.34999999999999987
initial condition: [10.192382587369362, 0.4326610475950516, 0.857685027320737, 6.870021710460025e-5]
last point: [10.204919765699357, 0.43232023926285995, 0.8578216235263332, 6.823357770280547e-5]
LSE: [-0.5555555555687532, -2.5305375685271327, -2.5295077854211194, -4.374128613263122]
_________________

->
index: 70
I0: -1.4096828282828373
U0: 0.34999999999999987
initial condition: [10.204919765699357, 0.43232023926285995, 0.8578216235263332, 6.823357770280547e-5]


initial condition: [10.450975172541353, 0.4257344868437888, 0.8604503767572081, 5.9813562323100774e-5]
last point: [10.463055810013497, 0.42541611429075976, 0.8605769400653026, 5.943394102901744e-5]
LSE: [-0.5555555555687434, -3.1995564315550067, -3.1985933067952064, -4.482030132003557]
_________________

->
index: 91
I0: -1.2925876767676885
U0: 0.34999999999999987
initial condition: [10.463055810013497, 0.42541611429075976, 0.8605769400653026, 5.943394102901744e-5]
last point: [10.475116383921069, 0.4250987274032579, 0.8607030643843968, 5.9057893040741326e-5]
LSE: [-0.5555555555687433, -3.2301615495011533, -3.2300528868871945, -4.487032319677626]
_________________

->
index: 92
I0: -1.287011717171729
U0: 0.34999999999999987
initial condition: [10.475116383921069, 0.4250987274032579, 0.8607030643843968, 5.9057893040741326e-5]
last point: [10.487157033449234, 0.4247823193167729, 0.8608287529252615, 5.868537291971776e-5]
LSE: [-0.5555555555687431, -3.257953616340371, -3.2641822251339923,

In [89]:
function firstdot( I0, U0, initial_condition_function )
    
    if I0 == startI
        println("FIRST ITERATION");flush(stdout)
        println("MOVE TO RIGHT")
        step_to_right = rangeI.step.hi
        range_to_right_border = range( I0 + step_to_right, endI, step = step_to_right )
    
        for (index, I0_right) in enumerate(range_to_right_border)
        
            if index == 1
                global initial_condition_right = initial_condition_function
            end
            println("->")
            print_output(index, I0_right, U0, initial_condition_right)
        
            parametres = SA[  τ, α, τ_D, J, U0, ΔU0, τ_y, β, xthr, ythr, I0_right  ]
        
            ds = ContinuousDynamicalSystem( model, initial_condition_right, parametres )
    
            tr = trajectory(ds, timefinal, Ttr = timefinal, Δt = 0.001;
                    diffeq = (alg = RK4(), adaptive = false, dense = false, dt = 0.001))
            initial_condition_right = tr[end, :]
        
            println("last point: $initial_condition_right");flush(stdout)
            λλ = lyapunovspectrum( ds, timefinal, Ttr = timefinal, Δt = 1;
                            diffeq = ( alg = RK4(), adaptive = false, dense = false, dt = 0.001 ))
            println("LSE: $λλ");flush(stdout)
            """
            this save output
            """
            open(file, "a") do io
                println(io, " $I0 $U0 $(λλ[1])")
            end
            separate()
        end
    end
end

firstdot (generic function with 1 method)